### The QLBS Model

이제 Markov Dicision Process의 framework를 사용해 3절에서 제시한 이산 시간 BSM을 재구성하고 일반화한다. 핵심 아이디어는 이산 시간에서의 위험 기간 가격 결정과 헤징을 가치함수를 가진 MDP문제로 이해할 수 있다는 것이다. 이 가치 함수를 옵션 매도자에 대한 음의 위험 조정 옵션 가격으로 정의했다는 점을 상기하자.

### 1. State Variables

주가 동학은 일반적으로 결정적 추세 항을 포함하므로 새로운 시간 변환 변수가 정상성을 갖게, 즉 비추세적이 되도록 상태 변수의 변경을 고려할 수 있다. 주어진 주가 프로세스 $S_t$에 대해 다음 관계에 의해서 새로운 변수 $X_t$를 정의함으로써 이를 달성할 수 있다.

$$X_t = -\left( \mu - \frac{\sigma^2}{2}\right)t + \log S_t$$

이 표현의 장점은 $S_t$가 Geometric Brownian Motion인 특수한 경우에서 분명히 볼 수 있다. 이 경우 다음을 얻는다.

$$dX_t = - \left( \mu - \frac{\sigma^2}{2}\right)dt + d\log S_t = \sigma d W_t$$

따라서 $S_t$의 실제 동학이 로그 정규분포일 때 $X_t$는 변동성 $\sigma$에 의해 크기 조정된 표준 브라운 운동이다. 주어진 MC 시나리오에서 $X_t$를 안다면 $S_t$의 상응하는 값은 다음 공식에 의해 주어진다.

$$S_t = e^{X_t + \left( \mu - \frac{\sigma^2}{2}\right)t}$$

$\{X_t\}_{t=0}^T$이 martingale, 즉 $\mathbb{E}[dX_t] = 0, \forall t$인 한 평균적으로 옵션의 생애 동안 포기값 $X_0$으로부터 너무 떨어지지 않을 것이다. 상태 변수 $X_t$는 추세 항을 갖는 주가 $S_t$와 달리 time-uniform이다. 그러나 관계식 $S_t = e^{X_t + \left( \mu - \frac{\sigma^2}{2}\right)t}$는 $S_t$의 비정상성 동학을 $X_t$의 정상성 동학에 매핑하고자 항상 사용될 수 있다. $X_t$의 martingale 속성은 주가의 가능한 미래 변동을 포착하고자 그리드가 너무 커서는 안 된다는 것을 의미하기 때문에 수치 그리드 근사에도 도움이 된다.

식 $X_t = -\left( \mu - \frac{\sigma^2}{2}\right)t + \log S_t$의 변수 변화와 그 역도 주가 동학이 GBM이 아닐 때 적용할 수 있다. 물론 이 경우 새로운 상태 변수 $X_t$기 일반적으로 Martingale이 되지는 않겠지만 최적화 작업의 비정상성과 상태 변수의 비정상성을 분리하는 데 본질적으로 유용하다.

### 2. Bellman Equations

3.2절에서 설정한 위험 최소화 절차를 MDP 문제의 언어로 다시 설명하는 것으로 시작한다. 특히 time dependent state variable인 $S_t$는 식 $S_t = e^{X_t + \left( \mu - \frac{\sigma^2}{2}\right)t}$을 사용해 time-homogeneous variable $X_t$로 표현횐다. 또한 $a_t = a_t(X_t)$의 표기법을 사용해 시간 동질 변수 $X_t$의 함수로 표현되는 행동을 나타낼 것이다. 주가로 표현한 행동 $ut = u_t(S_t)$는 다음의 대입을 통해 구한다. 

$$u_t (S_t) = a_t(X_t(S_t)) = a_t \left(\log S_t - \left( \mu - \frac{\sigma^2}{2}\right)t\right)$$